<a href="https://colab.research.google.com/github/jongbokhi/Kaggle_Competition/blob/main/synthetic_testdata_CTGAN_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Generate Synthetic Data by using CTGAN

In [ ]:
!pip install table_evaluator

In [ ]:
!pip install sdv

In [ ]:
!pip install CTGAN

In [5]:
from table_evaluator import TableEvaluator

import tensorflow as tf

from functools import partial
import itertools
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sdv.single_table import CTGANSynthesizer

In [6]:
train = pd.read_csv('/content/drive/MyDrive/kaggle_Mohs hardness prediction/train.csv').drop('id',axis=1)
test = pd.read_csv('/content/drive/MyDrive/kaggle_Mohs hardness prediction/test.csv').drop('id',axis=1)

In [10]:
train

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
0,100.0,0.841611,10.000000,4.800000,20.612526,11.088100,2.766000,1.732000,0.860000,0.496070,0.91457,6.0
1,100.0,7.558488,10.000000,4.800000,20.298893,12.040830,2.755000,1.631000,0.910000,0.492719,0.71760,6.5
2,76.0,8.885992,15.600000,5.600000,33.739258,12.086300,2.828000,1.788000,0.864000,0.481478,1.50633,2.5
3,100.0,8.795296,10.000000,4.800000,20.213349,10.948500,2.648000,1.626000,0.936000,0.489272,0.78937,6.0
4,116.0,9.577996,11.600000,4.800000,24.988133,11.824480,2.766000,1.682000,0.896000,0.492736,1.86481,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10402,128.0,7.558488,12.000000,4.000000,26.385218,11.330440,2.644000,1.631000,0.892000,0.496070,1.79607,4.0
10403,30.0,1.743160,10.000000,5.333333,20.766935,14.163933,3.090000,1.556667,0.866667,0.480390,0.81480,5.0
10404,196.0,30.920000,24.500000,5.500000,53.490297,10.074300,2.295000,1.545000,1.120000,0.469715,2.11540,1.8
10405,38.0,1.553160,12.666667,4.666667,26.621687,11.290033,2.743333,1.756667,0.980000,0.486507,0.77755,6.0


In [7]:
hardness_clf = pd.read_csv('/content/drive/MyDrive/kaggle_Mohs hardness prediction/hardness_class_pre.csv')

In [8]:
test['Hardness']= hardness_clf

In [9]:
test

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
0,884.0,121.420000,35.360000,5.280000,82.561240,9.370384,2.298000,1.907200,1.200000,0.461899,1.79459,2.55
1,90.0,9.931960,18.000000,5.600000,39.568056,12.086300,2.868000,1.652000,0.864000,0.476196,1.41194,2.55
2,116.0,7.767992,11.600000,4.800000,23.231818,11.023840,2.644000,1.794000,0.960000,0.499514,0.78834,5.75
3,100.0,9.107996,10.000000,4.800000,20.298893,12.086300,2.828000,1.662000,0.792000,0.495796,1.20466,5.75
4,55.0,4.030000,11.000000,4.000000,22.977675,11.280950,2.455000,1.750000,0.893333,0.476095,0.93456,5.75
...,...,...,...,...,...,...,...,...,...,...,...,...
6934,446.0,36.135392,9.636364,4.636364,19.689448,11.045515,2.768636,1.769697,0.871818,0.498702,0.11548,5.75
6935,36.0,3.550000,18.000000,4.000000,36.068500,8.236600,1.790000,2.055000,1.390000,0.499000,1.35045,3.75
6936,68.0,4.545328,11.333333,5.333333,22.688853,10.938358,2.876667,1.713333,0.916667,0.499074,0.73425,5.75
6937,40.0,2.334164,6.666667,4.000000,13.016128,12.700467,2.770000,1.476667,0.616667,0.663797,0.51227,4.75


In [ ]:
train.shape

(10407, 13)

In [ ]:
train = train.drop("id",axis=1)

##1. detect Metadata from Train dataset

In [19]:
from sdv.metadata import SingleTableMetadata
##Reference:
#https://docs.sdv.dev/sdv/single-table-data/data-preparation/single-table-metadata-api
#Official tutorial code: https://colab.research.google.com/drive/15iom9fO8j_gHg4-NlGkzWF5thMWStXwv?usp=sharing#scrollTo=OG7FYIwYHzFA

metadata = SingleTableMetadata()

metadata.detect_from_dataframe(data=test)

##2. Creating a Synthesizer and generating Synthetic data

In [11]:
#https://docs.sdv.dev/sdv/single-table-data/modeling/synthesizers/ctgansynthesizer

synthesizer = CTGANSynthesizer(metadata, epochs=2000)
synthesizer.fit(test)

In [12]:
#Save trained CTGAN model
synthesizer.save(filepath='/content/drive/MyDrive/kaggle_Mohs hardness prediction/CTGANsynthesizer3_test.pkl')

In [11]:
import pickle

file_path = '/content/drive/MyDrive/kaggle_Mohs hardness prediction/CTGANsynthesizer3_test.pkl'

with open(file_path, 'rb') as file:
     synthesizer_CTGAN= pickle.load(file)

In [12]:
#generate synthetic data
synthetic_data = synthesizer_CTGAN.sample(num_rows=7000)

In [13]:
synthetic_data.head()

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
0,70.778240,14.606624,17.224001,4.803010,22.724101,11.271884,2.836448,1.756145,0.836578,0.495172,1.94564,5.75
1,51.879155,7.870493,9.855053,4.778086,29.334930,10.624783,2.890899,1.645331,0.871085,0.490304,1.09270,5.75
2,100.811575,9.732696,14.052671,4.802812,24.133635,11.537482,2.737736,1.743407,0.916228,0.489478,0.81468,5.75
3,23.518162,4.239374,9.041666,3.985347,26.724741,10.964217,2.497472,1.702067,0.975969,0.492206,1.29664,3.77
4,45.967035,7.018599,4.543423,2.619503,0.000000,12.731302,2.717420,1.394007,0.491518,0.628572,0.67019,2.53


In [14]:
#Save synthetic data
synthetic_data.to_csv('/content/drive/MyDrive/kaggle_Mohs hardness prediction/sytheticdata_test2.csv', index=False)

In [15]:
#combine synthetic data with original training data

combined_train = pd.concat([train, synthetic_data], axis=0, ignore_index=False)

In [16]:
combined_train.shape

(17407, 12)

In [17]:
combined_train

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
0,100.000000,0.841611,10.000000,4.800000,20.612526,11.088100,2.766000,1.732000,0.860000,0.496070,0.91457,6.00
1,100.000000,7.558488,10.000000,4.800000,20.298893,12.040830,2.755000,1.631000,0.910000,0.492719,0.71760,6.50
2,76.000000,8.885992,15.600000,5.600000,33.739258,12.086300,2.828000,1.788000,0.864000,0.481478,1.50633,2.50
3,100.000000,8.795296,10.000000,4.800000,20.213349,10.948500,2.648000,1.626000,0.936000,0.489272,0.78937,6.00
4,116.000000,9.577996,11.600000,4.800000,24.988133,11.824480,2.766000,1.682000,0.896000,0.492736,1.86481,6.00
...,...,...,...,...,...,...,...,...,...,...,...,...
6995,213.902695,13.073992,32.850465,4.319821,70.073897,11.809836,2.775578,1.721191,0.855476,0.423836,4.11198,5.74
6996,90.232516,10.027510,10.725960,4.807430,22.513868,11.160481,2.832072,1.638274,0.879393,0.495610,0.87324,5.75
6997,97.492287,20.360022,15.132144,4.804041,27.020485,12.710222,2.761044,1.700066,0.825442,0.492706,1.37177,5.74
6998,30.533360,7.479039,3.994751,4.810271,5.987025,11.525085,2.052403,1.734979,1.256191,0.478537,0.77473,6.54


## 3 Evaluating Real vs. Synthetic Data

In [20]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    test,
    synthetic_data,
    metadata
)

Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 66/66 [00:01<00:00, 62.55it/s]

Overall Quality Score: 93.92%

Properties:
- Column Shapes: 91.79%
- Column Pair Trends: 96.04%


In [21]:
quality_report.get_visualization('Column Shapes')

In [22]:
from sdv.evaluation.single_table import get_column_plot

names = list(test.columns)

for name in names:

  fig = get_column_plot(
      real_data=test,
      synthetic_data=synthetic_data,
      column_name=name,
      metadata=metadata
      )

  fig.show()

